# Création des colonnes "quartier_imm" et "quartier_pers" - partie II. 2.

In [1]:
# import librairies
import pandas as pd
import requests, json
import geopandas
from shapely.geometry import Polygon, LineString, Point
from unidecode import unidecode
from urllib.parse import quote 
import io
import re

# configurer les options : voir toutes les colonnes et désactiver des avertissements de copie
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
# import données nettoyées
path = "/home/pa/Documents/M1/memoire/notebooks_donnees_rendu/"
df_1898 = pd.read_csv(path + "donnees_nettoyees.csv")

# 1. Création colonne quartier_imm

## 1.1. Création d'une la colonne qui indique "True" pour les lignes problématiques (= sur plusieurs quartiers)

In [3]:
quartiers_mult_bool = df_1898.quartiers_imm.str.match("\d{1,2}[-\/,_ ]\d{1,2}([-\/,_ ]\d{1,2})?([-\/,_ ]\d{1,2})?([-\/,_ ]\d{1,2})?([-\/,_ ]\d{1,2})?([-\/,_ ]\d{1,2})?")
df_1898['bool_quartiers_multiples']= quartiers_mult_bool

In [4]:
# statistiques sur le nombre de lignes problématiques
df_1898.bool_quartiers_multiples.value_counts() / len(df_1898)

False    0.645068
True     0.354932
Name: bool_quartiers_multiples, dtype: float64

## 1.2. Création d'autant de lignes que d'immeubles

In [5]:
# création sauvegarde
df_sauvegarde = df_1898.copy(deep=True)

In [6]:
# fonction pour transformer la colonne '10 à 14' en liste [10, 12, 14]

def split_number(number):
    try:
        beg, end = number.split('à')
        
        # on supprime les bis et les ter
        beg = re.sub(pattern=r"([0-9]{1,3})bis", repl=r"\1", string=beg, count=1)
        beg = re.sub(pattern=r"([0-9]{1,3})ter", repl=r"\1", string=beg, count=1)
        end = re.sub(pattern=r"([0-9]{1,3})bis", repl=r"\1", string=end, count=1)
        end = re.sub(pattern=r"([0-9]{1,3})ter", repl=r"\1", string=end, count=1)        
        
        beg, end = int(beg), int(end)
                
        # on crée la liste et on rajoute les numéros des immeubles
        numbers = []
        while beg <= end:
            numbers.append(beg)
            beg += 2
                        
        return numbers
    
    # si il s'agit d'un numéro seul, pas besoin de cette manipulation
    except:
        return [number]

    

In [7]:
# on applique cette fonction a num_imm
listes_numeros = df_1898['num_imm'].apply(split_number)
listes_numeros = listes_numeros.apply(pd.Series).stack()
listes_numeros = listes_numeros.droplevel(-1)

# j'essaye de convertir le type en int (entier)
listes_numeros.astype(int, errors='ignore')

/tmp/ipykernel_276260/294225800.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  listes_numeros = listes_numeros.apply(pd.Series).stack()


0           1
1         3.0
1         5.0
1         7.0
2        11.0
         ... 
83087      14
83088      16
83089      18
83090      20
83091      22
Length: 104819, dtype: object

In [8]:
# maintenant qu'on a procédé à la transformation, cette colonne
# contient le numero de l'immeuble
listes_numeros.name = "numero_imm"

In [9]:
df_1898 = df_1898.join(listes_numeros)

In [10]:
# le typage en tant qu'int est pas très efficace donc on utilise un regex pour transformer les "entier.0" en "entier"
df_1898["numero_imm"] = df_1898["numero_imm"].astype(str, errors='ignore')
df_1898["numero_imm"] = df_1898['numero_imm'].str.replace(r'([0-9]{1,3})\.0', r'\1', regex=True)

In [11]:
# on renomme la colonne "num_imm" qui contient plutot une liste
# des numéros
df_1898 = df_1898.rename({"num_imm": "liste_numeros_imm"}, axis=1, copy=False)

In [12]:
taille_initiale = len(df_sauvegarde)
taille_actuelle = len(df_1898)

print(f"Il y avait initialement {taille_initiale} lignes ; il y a maintenant {taille_actuelle} lignes qui correspondent à autant d'immeubles.")

Il y avait initialement 83092 lignes ; il y a maintenant 104827 lignes qui correspondent à autant d'immeubles.


In [13]:
# calcul du nombre de lignes problématiques après cette manipulation
total = len(df_1898)
pb = len(df_1898.loc[df_1898["bool_quartiers_multiples"] == True])
ok = len(df_1898.loc[df_1898["bool_quartiers_multiples"] == False])
pb_perc = round((pb/total)*100, 2)

print(f"Sur {total} adresses totales, il y en a {pb_perc} % de problématiques c'est-à-dire qui peuvent être dans plusieurs quartiers.")

Sur 104827 adresses totales, il y en a 35.33 % de problématiques c'est-à-dire qui peuvent être dans plusieurs quartiers.


## 1. 3. Application du géocodeur aux adresses des immeubles

In [14]:
def mass_search(df, col_numero, col_type, col_nom, 
                *, 
                suffix, col_ville=None):
    """
    appelle le géocodeur https://api-adresse.data.gouv.fr
    et renvoie le dataframe avec de nouvelles informations :
    latitude, longitude, ville et type de résultat de l'adresse trouvée
    
    Paramètres :
      df :
        le dataframe en entrée
      col_numero :
      col_type :
      col_nom :
        le nom des 3 colonnes qui contiennent le numéro, le type
        de voie et le nom de la rue qui seront utilisées par le
        géocodeur
      col_ville :
        ville dans laquelle le géocodeur va chercher, par défaut Paris
      suffix :
        est utilisé pour nommer les 4 colonnes en sorties
        par exemple suffix="imm" fera comme résultat :
        lat_imm, lng_imm, result_type_imm et result_city_imm
    
    """
    
    # constantes internes
    dunder_indexname = f'__index_{suffix}__'
    dunder_filename = '__search_data__.csv'

    # cherche le nom de l'index ; en créé un si inexistant
    if df.index.name is None:
        df.index.name = dunder_indexname
    indexname = df.index.name
    
    # vérifier que l'index est unique
    df.reset_index(inplace=True)

    # créer le dataframe qu'on enverra au géocodeur
    search_data = df[[col_numero, col_type, col_nom]]

    # si on a pas de col_ville
    col_ville = col_ville or 'Paris'
    # on remplit la colonne ville
    if col_ville in df.columns:
        # si la colonne ville est une colonne existante
        search_data['city'] = df[col_ville]
    else:
        # sinon, rajouter la constante (en général Paris)
        search_data['city'] = col_ville
        
    # sauvegarder les informations pour les envoyer au géocodeur
    search_data.to_csv(dunder_filename, index=False)
    with open(dunder_filename) as feed:
        result = requests.post(
            "https://api-adresse.data.gouv.fr/search/csv/", 
            files={'data': feed},
            data={'columns': search_data.columns})
    # sauvegarder le résultat
    result = pd.read_csv(io.StringIO(result.text))
    
    # on ne garde que ce qui nous intéresse
    result = (result[['latitude', 'longitude', 'result_type', 'result_city']]
              # on renomme pour respecter les suffixes
              .rename(columns={
                          'latitude': f'lat_{suffix}',
                          'longitude': f'lng_{suffix}',
                          'result_type': f'result_type_{suffix}',
                          'result_city': f'result_city_{suffix}',
                      }))
    
    # on fusionne le df initial et celui qu'on a obtenu du géocodeur
    # et on restore l'index initial
    return df.merge(result, left_index=True, right_index=True).set_index(indexname)


In [15]:
# on applique la fonction et on la chronometre
import time
beg = time.time()
geoloc = mass_search(df_1898, 'numero_imm', 'type_voie_imm', 'nom_voie_imm', suffix="imm")
print(time.time()-beg)

79.01582932472229


In [16]:
# on transforme le geoloc en effectivement un geodataframe geopandas
geoloc['coord_imm'] = [Point(xy) for xy in zip(geoloc.lng_imm, geoloc.lat_imm)] 
geoloc = geopandas.GeoDataFrame(geoloc, geometry=geoloc.coord_imm) 
type(geoloc)

geopandas.geodataframe.GeoDataFrame

### 1. 4. Test d'efficacité de la requête du géocodeur

In [17]:
# on vérifie que notre dataframe d'input a la même longueur que
# celui d'output
difference = len(df_1898) - len(geoloc)
print(difference)

0


In [18]:
depart = len(df_1898)
df_erreurs = geoloc[geoloc["result_type_imm"] != 'housenumber']
erreurs = len(df_erreurs)
# on arrondit à deux chiffres après la virgule
pourcent_erreurs = round((erreurs / depart)*100, 2)

print(f'Pour {depart} adresses initiales, il y a {erreurs} adresses qui ne sont pas une adresse avec un numéro, soit {pourcent_erreurs}%.')

Pour 104827 adresses initiales, il y a 14412 adresses qui ne sont pas une adresse avec un numéro, soit 13.75%.


In [19]:
depart = len(geoloc)
gdf_erreurs_importantes = geoloc.loc[(geoloc["result_type_imm"] != 'housenumber') & (geoloc["bool_quartiers_multiples"] == True)]
erreurs = len(gdf_erreurs_importantes)
# idem
pourcent_erreurs_importantes = round((erreurs / depart)*100, 2)

print(f'Pour {depart} lignes initiales, il y a {erreurs} adresses qui n\'ont pas de numéro et dont on va utiliser les coordonnées du géocodeur, soit {pourcent_erreurs_importantes}% du total.')

Pour 104827 lignes initiales, il y a 4571 adresses qui n'ont pas de numéro et dont on va utiliser les coordonnées du géocodeur, soit 4.36% du total.


## 1.5. La colonne quartier_imm

In [20]:
# je vais chercher le gdf quartiers (source : open data Paris)
# remarque : il faut tous les fichiers "quartier_paris" pour que ça marche
csv_folder = "/home/pa/Documents/M1/memoire/TAIS/donnees/vecteurs/"
gdf_quartiers = geopandas.read_file(csv_folder+"quartier_paris.shp", encoding='utf-8')

# et j'adapte le type de la colonne quartier
gdf_quartiers['c_qu'] = gdf_quartiers["c_qu"].astype(int)

# je rajoute la colonne nécessaire
df_1898['quartier_in_quartiers_imm'] = pd.Series(dtype='bool')

### 1. 5. 1 Cas problématiques : sjoin

In [21]:
# je fais un sous-dataframe qui ne contient que les adresses problématiques
gdf_lignes_problematiques = geoloc[geoloc["bool_quartiers_multiples"] == True]

In [22]:
# j'assigne les quartiers avec un sjoin pour les cas problématiques
gdf_lignes_problematiques = gdf_quartiers.sjoin(gdf_lignes_problematiques, predicate='contains', how='right')

/home/pa/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:2061: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [23]:
# je renomme la colonne ainsi obtenue pour qu'elle rentre dans les standards
gdf_lignes_problematiques.rename(columns = {'c_qu':'quartier_imm'}, inplace = True)

### 1. 5. 2. quartiers non problématiques

In [24]:
# création sous-dataframe
gdf_non_pb = geoloc.loc[geoloc['bool_quartiers_multiples'] == False]
# on recupère simplement l'information de "quartiers_imm"
gdf_non_pb["quartier_imm"] = gdf_non_pb["quartiers_imm"]

### 1. 5. 3. re-rassemblage des 2 sous-dataframes

In [25]:
# je vais re-join ces df
gdf_final = pd.concat([gdf_non_pb, gdf_lignes_problematiques])

In [26]:
# et nous ne gardons
gdf_final = gdf_final[[
    'index_left', 'code_voie_quartiers_imm', 'code_voie_imm',
    'liste_numeros_imm', 'numero_imm', 'type_voie_imm', 'nom_voie_imm',
    'nb_imm', 'quartier_imm', 'quartiers_imm', 'bool_quartiers_multiples',
    'complement_nom_voie_imm', 'precisions_imm', 'num_pers', 
    'type_voie_pers', 'nom_voie_pers',
    'nom_pers', 'prenom_pers', 'civilite_pers', 'code_pers', 
    'desamb_pers', 'ville_pers', 'departement_pers', 'ger', 
    'org', 'org_hom', 'code_org', 'code_secteur_activite_org',
    'rem', 'entree_imm', 'Entree=0.1_Parisien=1.0_Nonparisien=0.0', 
    'lat_imm', 'lng_imm', 'coord_imm',
    'result_type_imm', 'result_city_imm', 'geometry']]

## 1. 6. Controle des longueurs

In [27]:
print(f"Longueur du dataframe de départ : {len(df_1898)}")
print(f"Longueur du dataframe après application géocodeur : {len(geoloc)}")

print(f"Longueur du dataframe des adresses problématiques : {len(gdf_lignes_problematiques)}")
print(f"Longueur du dataframe des adresses non problématiques : {len(gdf_non_pb)}")
print(f"Somme de la longueur du dataframe des adresses problématiques et non problématiques : {len(gdf_lignes_problematiques) + len(gdf_non_pb)}")

print(f"Longueur du dataframe final : {len(gdf_final)}")

Longueur du dataframe de départ : 104827
Longueur du dataframe après application géocodeur : 104827
Longueur du dataframe des adresses problématiques : 37033
Longueur du dataframe des adresses non problématiques : 67794
Somme de la longueur du dataframe des adresses problématiques et non problématiques : 104827
Longueur du dataframe final : 104827


In [28]:
# ce qui est cohérent.

## 1. 7. Création colonne quartier_in_quartiers

In [29]:
len(gdf_final.loc[gdf_final["quartier_imm"] == "x"])

20

In [30]:
# je remplace ces quelques donnees par "0" pour pouvoir changer de type
gdf_final["quartier_imm"].replace(to_replace="x", value=0, inplace=True, )

In [31]:
gdf_final["quartier_imm"] = gdf_final["quartier_imm"].astype(float).astype('Int64')

In [32]:
def fonct_quartier_in_quartiers(row):
    return str(row.quartier_imm) in row.quartiers_imm

gdf_final['quartier_in_quartiers_imm'] = gdf_final.apply(fonct_quartier_in_quartiers, axis=1)

### 6. Quelques indicateurs de qualité

In [33]:
# 1. fréquence de la présence d'un quartier
quartier_nul = gdf_final['quartier_imm'].isnull().sum()
total_adresses = len(gdf_final)
pourcentage_quartier_nul = round((((total_adresses-quartier_nul) / total_adresses)*100), 2)

print(f"Nous ne trouvons pas de quartier pour {quartier_nul} immeubles, ce qui signifie que nous trouvons un quartier pour {pourcentage_quartier_nul} % des immeubles.")

Nous ne trouvons pas de quartier pour 1028 immeubles, ce qui signifie que nous trouvons un quartier pour 99.02 % des immeubles.


In [34]:
# 2. est-ce que les quartiers trouves sont plausibles = dans les quartiers possibles ?
gdf_final.quartier_in_quartiers_imm.value_counts()/len(gdf_final)*100

True     98.075877
False     1.924123
Name: quartier_in_quartiers_imm, dtype: float64

In [35]:
# 3. d'ou viennent ces erreurs ?
gdf_erreurs_quartier = gdf_final.loc[gdf_final["quartier_in_quartiers_imm"]==False]

In [36]:
# - dans certains cas c'est les quelques adresses etranges qui ont pas de quartier renseigne dans les donnees
len(gdf_erreurs_quartier.loc[gdf_erreurs_quartier.quartier_imm == 0])

20

In [37]:
# apparemment le géocodeur trouve toujours quelque chose (après pas forcément la bonne adresse)
len(gdf_erreurs_quartier.loc[gdf_erreurs_quartier["coord_imm"] == (None, None, None, None, None)])

0

In [38]:
# certains cas sont des erreurs de géocodeur, dues à l'évolution des noms
len(gdf_erreurs_quartier.loc[(gdf_erreurs_quartier.type_voie_imm == "avenue") & (gdf_erreurs_quartier.nom_voie_imm == "ANTIN")])

52

In [39]:
# on trouve beaucoup les memes rues dans ces erreurs
gdf_erreurs_quartier.loc[(gdf_erreurs_quartier.quartier_imm != 0) & (gdf_erreurs_quartier.nom_voie_imm != "ANTIN")].head(50)

,index_left,code_voie_quartiers_imm,code_voie_imm,liste_numeros_imm,numero_imm,type_voie_imm,nom_voie_imm,nb_imm,quartier_imm,quartiers_imm,bool_quartiers_multiples,complement_nom_voie_imm,precisions_imm,num_pers,type_voie_pers,nom_voie_pers,nom_pers,prenom_pers,civilite_pers,code_pers,desamb_pers,ville_pers,departement_pers,ger,org,org_hom,code_org,code_secteur_activite_org,rem,entree_imm,Entree=0.1_Parisien=1.0_Nonparisien=0.0,lat_imm,lng_imm,coord_imm,result_type_imm,result_city_imm,geometry,quartier_in_quartiers_imm
__index_imm__,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5951,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,1,1,boulevard,BEAUMARCHAIS,1,15,11-13-42-43,True,0,0,4,r.,St-Martin,Quainon,0,Vve,7501.0,0,Paris,0,0,0,0,0.0,0,0,0,1.0,48.853922,2.368594,POINT (2.368594 48.853922),housenumber,Paris,POINT (2.36859 48.85392),False
5952,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,3,3,boulevard,BEAUMARCHAIS,1,15,11-13-42-43,True,0,0,72,r.,Rivoli,0,0,Vve,0.0,0,Paris,0,0,Frager et Michel,Frager et Michel,516.0,0,0,0,1.0,48.854052,2.368603,POINT (2.368603 48.854052),housenumber,Paris,POINT (2.36860 48.85405),False
5953,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,5,5,boulevard,BEAUMARCHAIS,1,15,11-13-42-43,True,0,0,9,r.,du 4 Septembre,Aucoc,0,0,40695.0,0,Paris,0,0,0,0,0.0,0,0,0,1.0,48.854203,2.368612,POINT (2.368612 48.854203),housenumber,Paris,POINT (2.36861 48.85420),False
5954,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,7 à 9,7,boulevard,BEAUMARCHAIS,2,15,11-13-42-43,True,0,0,26,fg,Poissonnière,Durenne,0,Hiers,27662.0,blanc vs heirs,Paris,0,gér. Henry,0,0,0.0,0,0,0,1.0,48.854354,2.368581,POINT (2.368581 48.854354),housenumber,Paris,POINT (2.36858 48.85435),False
5954,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,7 à 9,9,boulevard,BEAUMARCHAIS,2,15,11-13-42-43,True,0,0,26,fg,Poissonnière,Durenne,0,Hiers,27662.0,blanc vs heirs,Paris,0,gér. Henry,0,0,0.0,0,0,0,1.0,48.854481,2.368551,POINT (2.368551 48.854481),housenumber,Paris,POINT (2.36855 48.85448),False
5955,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,11,11,boulevard,BEAUMARCHAIS,1,15,11-13-42-43,True,0,0,1,boul.,Voltaire,Pinguet,0,0,8712.0,0,Paris,0,0,0,0,0.0,0,0,0,1.0,48.854546,2.368535,POINT (2.368535 48.854546),housenumber,Paris,POINT (2.36854 48.85455),False
5956,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,13,13,boulevard,BEAUMARCHAIS,1,15,11-13-42-43,True,0,0,94,av.,Malakoff,Pastureau,0,0,10138.0,0,Paris,0,0,0,0,0.0,0,0,0,1.0,48.854677,2.368487,POINT (2.368487 48.854677),housenumber,Paris,POINT (2.36849 48.85468),False
5957,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,15,15,boulevard,BEAUMARCHAIS,1,15,11-13-42-43,True,0,0,15,boul.,Beaumarchais,Brientin,0,0,35889.0,0,Paris,0,0,0,0,0.0,0,0,0,1.0,48.854926,2.368445,POINT (2.368445 48.854926),housenumber,Paris,POINT (2.36844 48.85493),False
5958,56.0,B.101 BEAUMARCHAIS (Boulev.) 11_13_42_43,B.101,17,17,boulevard,BEAUMARCHAIS,1,15,11-13-42-43,True,0,0,0,0,0,Bataille,0,0,NaN,0,Bonfruit,Seine-et-Marne,0,0,0,0.0,0,0,0,0.0,48.855045,2.368412,POINT (2.368412 48.855045),housenumber,Paris,POINT (2.36841 48.85504),False


***

# 2. Création colonne quartier du domicile du/de la proprietaire

## 2. 1. Nettoyage de "type_voie_pers" pour qu'elle soit en toutes lettres

In [40]:
# je commence par harmoniser les type_voie_pers qui ont souvent des abbréviations
dico_abbr = {"r.": "rue", "r": "rue", "rue": "rue",
             "rte": "route", "Rte": "route", "route": "route",
             "boul.": "boulevard", "boul /": "boulevard", "b.": "boulevard",
             "av.": "avenue", "eav.": "avenue", "Av.": "avenue",
             "pl.": "place", "place": "place", "pi.": "place",
             "imp.": "impasse",
             "pass": "passage", "pass.": "passage",
             "fg": "rue du faubourg", "fg.": "rue du faubourg", "faub.": "rue du faubourg",
             "villa": "villa", "Villa": "villa", "v.": "villa",
             "cité": "cité", "Cité": "cité", "cite": "cité",
             "quai": "quai", "q.": "quai", "Quai": "quai",
             "ch.": "chemin", 
             "cours": "cours", "Cours": "cours", "Cours ": "cours",
             "rond-point": "rond-point",
             "Chaussée": "rue de la Chaussée",
             "allée": "allée",
             "hameau": "hameau",
             "0": "0"}

In [41]:
# 1. tests
serie = gdf_final.type_voie_pers.map(dico_abbr)

nombre_erreurs_voies = serie.isna().sum()
total = len(gdf_final)
pourcent_erreurs_voies = round((nombre_erreurs_voies/total)*100, 2)

print(f"Nous avons {nombre_erreurs_voies} erreurs dans les voies, ce qui représente {pourcent_erreurs_voies}% d'erreurs.")

Nous avons 384 erreurs dans les voies, ce qui représente 0.37% d'erreurs.


In [42]:
# 2. application
gdf_final["type_voie_pers"] = gdf_final.type_voie_pers.map(dico_abbr)

## 2. 1. Application fonction gécodeur

In [43]:
# on applique la fonction et on la chronomètre
import time
beg = time.time()
geoloc_2 = mass_search(gdf_final, 'num_pers', 'type_voie_pers',
                     'nom_voie_pers', suffix="pers")
print(time.time()-beg)

254.6578733921051


In [44]:
len(geoloc_2)

104827

In [45]:
# conversion en geodataframe
geoloc_2['coord_pers'] = [Point(xy) for xy in zip(geoloc_2.lng_pers, geoloc_2.lat_pers)] 
geoloc_2 = geopandas.GeoDataFrame(geoloc_2, geometry=geoloc_2.coord_pers)
type(geoloc_2)

geopandas.geodataframe.GeoDataFrame

In [46]:
len(geoloc_2)

104827

### 2.2 Stats et application du sjoin quartier

In [47]:
# longueur des df : initial, après 1er traitement, après 2e traitement
print(len(df_1898))
print(len(geoloc))
print(len(geoloc_2))

104827
104827
104827


In [48]:
# création sous-df avec une adresse (qui s'appelera particulier pour le moment)
geoloc_2_pers = geoloc_2.loc[(geoloc_2.num_pers != "0") & (geoloc_2.type_voie_pers != "0") & (geoloc_2.nom_voie_pers != "0")]

In [49]:
# création sous-df avec les organisations
geoloc_2_org = geoloc_2.loc[(geoloc_2.num_pers == "0") | (geoloc_2.type_voie_pers == "0") | (geoloc_2.nom_voie_pers == "0")]

In [50]:
print(f"Longueur du dataframe total : {len(geoloc_2)}")
print(f"Longueur du dataframe des particuliers : {len(geoloc_2_pers)}")
print(f"Longueur du dataframe organisations : {len(geoloc_2_org)}")
print(f"Différence entre la longueur du dataframe total et la somme des longueurs des 2 sous-df : {len(geoloc_2) - len(geoloc_2_pers) - len(geoloc_2_org)}")
# > c'est cohérent

Longueur du dataframe total : 104827
Longueur du dataframe des particuliers : 75695
Longueur du dataframe organisations : 29132
Différence entre la longueur du dataframe total et la somme des longueurs des 2 sous-df : 0


In [51]:
# est-ce qu'on trouve bien des numéros pour les adresses des particuliers ?
depart = len(geoloc)
total_pers = len(geoloc_2_pers)
df_erreurs = geoloc_2_pers[geoloc_2_pers["result_type_pers"] != 'housenumber']
erreurs = len(df_erreurs)
pourcent_erreurs_dans_pers = round((erreurs / total_pers)*100, 2)
pourcent_erreurs_total = round((erreurs / depart)*100, 2)

print(f'Pour {depart} lignes initiales, on trouve {pourcent_erreurs_dans_pers}% d\'adresses qui n\'ont pas de numéro par rapport au nombre d\'adresses qui appartiennent à un particulier ; et {pourcent_erreurs_total} % d\'adresses qui n\'ont pas de numéro par rapport au nombre d\'adresse total.')

Pour 104827 lignes initiales, on trouve 11.76% d'adresses qui n'ont pas de numéro par rapport au nombre d'adresses qui appartiennent à un particulier ; et 8.49 % d'adresses qui n'ont pas de numéro par rapport au nombre d'adresse total.


In [52]:
geoloc_2_pers = geoloc_2_pers.rename(columns={"index_left": "index"})

In [53]:
# application du sjoin pour relier les adresses aux quartiers
gdf_2_pers = gdf_quartiers.sjoin(geoloc_2_pers, predicate='contains', how='right')
gdf_2_pers.rename(columns = {'c_qu':'quartier_pers'}, inplace = True)

/home/pa/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:2061: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [54]:
geoloc_2_org['quartier_pers'] = pd.Series(dtype='int')
geoloc_2_org["quartier_pers"] = geoloc_2_org["quartier_pers"].fillna(0)

In [55]:
gdf_2_final = pd.concat([gdf_2_pers, geoloc_2_org])

In [56]:
# nettoyage des colonnes qu'on veut pas de gdf_quartiers
gdf_2_final = gdf_2_final[[
    'index_left', 'index', 'code_voie_quartiers_imm', 'code_voie_imm',
    'liste_numeros_imm', 'numero_imm', 'type_voie_imm', 'nom_voie_imm',
    'nb_imm', 'quartier_imm', 'quartiers_imm', 'quartier_in_quartiers_imm',
    'bool_quartiers_multiples', 'complement_nom_voie_imm',
    'precisions_imm', 'num_pers', 'type_voie_pers', 'nom_voie_pers',
    'quartier_pers',
    'nom_pers', 'prenom_pers', 'civilite_pers', 'code_pers',
    'desamb_pers', 'ville_pers', 'departement_pers', 'ger',
    'org', 'org_hom', 'code_org', 'code_secteur_activite_org',
    'rem', 'entree_imm', 'Entree=0.1_Parisien=1.0_Nonparisien=0.0',
    'lat_imm', 'lng_imm', 'coord_imm', 'result_type_imm',
    'result_city_imm', 'geometry',
    'lat_pers', 'lng_pers', 'coord_pers',
    'result_type_pers', 'result_city_pers']]

In [57]:
print(len(df_1898))
print(len(geoloc))
print(len(geoloc_2))
print(len(geoloc_2_pers))
print(len(geoloc_2_org))
print(len(gdf_2_final))

104827
104827
104827
75695
29132
104827


In [58]:
# note : les 0 sont les organisations qui n'ont donc pas d'adresse
gdf_2_final["quartier_pers"].value_counts()

0.0     29132
32.0     2335
31.0     2034
34.0     1761
62.0     1730
        ...  
54.0      307
74.0      283
1.0       277
16.0      230
47.0      201
Name: quartier_pers, Length: 81, dtype: int64

In [59]:
gdf_2_final["quartier_pers"] = gdf_2_final["quartier_pers"].astype(float).astype('Int64')

In [60]:
# 1. est-ce qu'on trouve souvent un quartier ?
total = len(gdf_2_final)

gdf_ss_quartier_av_adr = gdf_2_final.loc[(gdf_2_final.num_pers != '0') & (gdf_2_final.type_voie_pers != '0') & (gdf_2_final.nom_voie_pers != '0') & (gdf_2_final.quartier_pers.isna() == True)]

nb_sans_quartier_av_adr = len(gdf_ss_quartier_av_adr)

pourc_ss_quartier_av_adresses = round((nb_sans_quartier_av_adr/total)*100, 2)

print(f"On a {pourc_ss_quartier_av_adresses} % de lignes qui ont une adresse complète pour le domicile du propriétaire mais pas de quartier par rapport au total d'adresses.")

On a 3.92 % de lignes qui ont une adresse complète pour le domicile du propriétaire mais pas de quartier par rapport au total d'adresses.


In [61]:
# nous n'avons pas de ligne qui ont une adresse de domicile complète mais un quartier de 0.
print(len(gdf_2_final.loc[(gdf_2_final.num_pers != '0') & (gdf_2_final.type_voie_pers != '0') & (gdf_2_final.nom_voie_pers != '0') & (gdf_2_final.quartier_pers == '0')]))
print(len(gdf_2_final.loc[(gdf_2_final.num_pers != '0') & (gdf_2_final.type_voie_pers != '0') & (gdf_2_final.nom_voie_pers != '0') & (gdf_2_final.quartier_pers == 0)]))

0
0


***

## 3. Création colonnes "meme_adresse" et "meme_quartier" et de statistiques

In [62]:
# création nouvel index propre
gdf_2_final['new_index'] = range(0, len(geoloc_2))
gdf_2_final = gdf_2_final.set_index('new_index')

In [63]:
# on nettoie le nom des rues

def nettoyage_nom_voie(row):
    # minuscule + sans accent
    gdf_2_final["nom_voie_imm"][row.name] = unidecode(str(row.nom_voie_imm).lower())
    gdf_2_final["nom_voie_pers"][row.name] = unidecode(str(row.nom_voie_pers).lower())
            
gdf_2_final.apply(nettoyage_nom_voie, axis=1)

new_index
0         None
1         None
2         None
3         None
4         None
          ... 
104822    None
104823    None
104824    None
104825    None
104826    None
Length: 104827, dtype: object

In [64]:
len(gdf_2_final)

104827

In [65]:
gdf_2_final['meme_adresse'] = (gdf_2_final.numero_imm == gdf_2_final.num_pers) & (gdf_2_final.type_voie_imm == gdf_2_final.type_voie_pers) & (gdf_2_final.nom_voie_imm == gdf_2_final.nom_voie_pers)

In [66]:
gdf_2_final['meme_adresse'].value_counts()

False    86704
True     18123
Name: meme_adresse, dtype: int64

In [67]:
gdf_2_final['meme_adresse'].value_counts() / len(geoloc_2)*100

False    82.711515
True     17.288485
Name: meme_adresse, dtype: float64

In [68]:
gdf_2_final['meme_quartier'] = gdf_2_final['quartier_imm'] == gdf_2_final['quartier_pers']

In [69]:
gdf_2_final['meme_quartier'].value_counts(dropna=False)

False    68910
True     31034
<NA>      4883
Name: meme_quartier, dtype: Int64

In [70]:
gdf_2_final['meme_quartier'].value_counts(dropna=False)/len(geoloc_2)*100

False    65.736881
True     29.604968
<NA>      4.658151
Name: meme_quartier, dtype: Float64

In [71]:
# je supprime geometry qui est deja stocke dans coord_pers et qui cause des pb pour rouvrir le doc
gdf_2_final = gdf_2_final.drop(['geometry'], axis=1)

In [72]:
# j'exporte mes données ; je vais les utiliser dans le notebook suivant
gdf_2_final.to_csv('donnees_apres_creation colonnes.csv')

***